# osiris

![img](https://dm2301files.storage.live.com/y4mmRC1xelS6Y6MEqUnZ-k2vjpADHpo6UMZAaZWROunr9-Ml5FYDlZ6WMxCGedy7NDhwDpusZdF5E1oLR5Qn6momydHe7tYUOMwNeFeGW7pUWkBjGPSnZp2sacYWs9IKkose6xjhSySL_v2tbfItRI7T_Pw_Tayhaa2F_vrwW6ucyr6WPa6s9DWH_if9Y5Y3yAU?width=375&height=250&cropmode=none)


osiris is a Python data processing and analysis environment for data-based computational conflict forecasting using very large datasets and graph-based methods and models and visualization, powered by scalable graph databases.

You can use osiris to analyze causal chains and networks of confict and violence around the world from realtime-updated, [automatically-encoded political event data](https://parusanalytics.com/eventdata/papers.dir/Schrodt_Yonamine_NewDirectionsInText.pdf) from projects like GDELT. This notebook gives an overview of the osiris project, the [GDELT project](https://www.gdeltproject.org/) data that osiris uses, how to import political event data using osiris either from the GDELT file server or from Google BigQuery, how to visualize and analyze it using Python, and how to load it into a TigerGraph graph server instance to efficiently run graph-centric queries on it to retrieve vertex-edge event data that can then be further analyzed.

## Notebook Environment Setup

In [2]:
import os, sys
# Check if running inside Colab or Kaggle
IN_COLAB = 'COLAB_GPU' in os.environ
IN_KAGGLE = 'KAGGLE_KERNEL_RUN_TYPE' in os.environ
IN_HOSTED_NB = IN_COLAB or IN_KAGGLE
os.environ['IN_HOSTED_NB'] = str(IN_HOSTED_NB)

OS_NAME = sys.platform.upper()
if OS_NAME in ['LINUX', 'DARWIN'] and IN_HOSTED_NB:
  import subprocess
  print('Installing osiris from GitHub...')
  print(subprocess.run('if [ -d "osiris" ]; then rm -Rf osiris; fi', text=True, shell=True, check=True, capture_output=True).stdout)
  print(subprocess.run('git clone https://github.com/allisterb/osiris --recurse-submodule', text=True, shell=True, check=True, capture_output=True).stdout)
  print(subprocess.run('cd osiris && ./install', text=True, shell=True, check=True, capture_output=True).stdout)

# If we're not in a hosted nb env assume we're running Jupyter from the osiris project directory root
OSIRIS_PATH = '..' if not IN_HOSTED_NB else 'osiris'

# Import the osiris code and set the runtime env. 
sys.path.append(os.path.join(OSIRIS_PATH, 'osiris'))
sys.path.append(os.path.join(OSIRIS_PATH, 'ext'))
from osiris_global import set_runtime_env
set_runtime_env(interactive_nb=True)

## GDELT Event Data

*From the  [GDELT project](https://www.gdeltproject.org/) website*:
>The GDELT Project is a realtime network diagram and database of global human society for open research.
![gf](https://www.gdeltproject.org/images/spinningglobe.gif)

>The GDELT Project is an initiative to construct a catalog of human societal-scale behavior and beliefs across all countries of the world, connecting every person, organization, location, count, theme, news source, and event across the planet into a single massive network that captures what's happening around the world, what its context is and who's involved, and how the world is feeling about it, every single day.

The GDELT [event data](http://data.gdeltproject.org/documentation/GDELT-Event_Codebook-V2.0.pdf) contains hundreds of millions of automatically coded events extracted from news stories daily using NLU methods and models. Each event data row contains the following fields:
1. *Actors*: Humans or organizations or states which initiate and are the target of event actions. Actors may have geographic information but not temporal. An event references exactly 2 actors: Actor1 and Actor2.
2. *Actions*: Codes and other information which describe each event. Actions have both temporal and spatial attributes: an event time plus some geo information like latitude / longitude. Actors and actions naturally form graphs with directed edges connecting Actor1-->Action-->Actor2. An Actor-Action edge may contain attributes like the event time and a complementary reverse edge to make querying easier e.g. Actor1----event1_date---->Action1----event1_date--->Actor2----->event2_date----->Action2----event2_date---->Actor3 
3. *SourceURL*: a URL that locates the *story* from which the event data was extracted.

osiris can extract data directly from the GDELT file server. The advantage of this method is that you don't need to have any special credentials or server access (remember we're interested *open-source* indicators.). All the data is downloaded directly to your client machine or notebook environment.

### Importing GDELT data from file server

osiris uses *DataSource* classes to manage importing tabular data. 

In [3]:
# Import data directly from GDELT file server
from data.gdelt import DataSource
import pandas as pd
gdelt = DataSource()

In [4]:
# Get event data for a 1 week period
events = gdelt.import_data('events', 'Apr-14-2022', 'Apr-20-2022')

Importing GDELT events data for 7 day(s) from 04-14-2022 to 04-20-2022...


Import GDELT events data:   0%|          | 0/7 [00:00<?, ?day/s]

Importing GDELT events data for 7 day(s) from 04-14-2022 to 04-20-2022 completed in 71.06 s.


About a week's worth of event data in 2022 consists of about 700K events takes up about 340MB RAM.

In [5]:
events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 707186 entries, 0 to 125669
Data columns (total 62 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   GLOBALEVENTID          707186 non-null  int64  
 1   SQLDATE                707186 non-null  int64  
 2   MonthYear              707186 non-null  int64  
 3   Year                   707186 non-null  int64  
 4   FractionDate           707186 non-null  float64
 5   Actor1Code             640700 non-null  object 
 6   Actor1Name             640700 non-null  object 
 7   Actor1CountryCode      408112 non-null  object 
 8   Actor1KnownGroupCode   9610 non-null    object 
 9   Actor1EthnicCode       3423 non-null    object 
 10  Actor1Religion1Code    10452 non-null   object 
 11  Actor1Religion2Code    2561 non-null    object 
 12  Actor1Type1Code        296023 non-null  object 
 13  Actor1Type2Code        19713 non-null   object 
 14  Actor1Type3Code        495 non-null 

In [6]:
events

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,1039300890,20210414,202104,2021,2021.2849,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220414013000,https://thequietus.com/articles/31396-tony-all...
1,1039300891,20210414,202104,2021,2021.2849,NZL,NEW ZEALAND,NZL,NaN,NaN,...,4,"Brisbane, Queensland, Australia",AS,AS04,154654,-27.5000,153.0170,-1561728,20220414013000,https://www.sen.com.au/news/2022/04/13/ayrton-...
2,1039300892,20210414,202104,2021,2021.2849,USA,LAWRENCE,USA,NaN,NaN,...,3,"Rockville Centre, New York, United States",US,USNY,NaN,40.6587,-73.6412,962765,20220414013000,http://www.5tjt.com/what-rockville-center-has-...
3,1039300893,20210414,202104,2021,2021.2849,USA,LAWRENCE,USA,NaN,NaN,...,2,"New York, United States",US,USNY,NaN,42.1497,-74.9384,NY,20220414013000,http://www.5tjt.com/what-rockville-center-has-...
4,1039300894,20210414,202104,2021,2021.2849,USAEDU,IOWA,USA,NaN,NaN,...,2,"Iowa, United States",US,USIA,NaN,42.0046,-93.2140,IA,20220414013000,https://www.thegazette.com/health-care-medicin...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125665,1040383216,20220420,202204,2022,2022.3014,cre,CREE,NaN,NaN,cre,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220420234500,https://www.cjvr.com/2022/04/20/first-nations-...
125666,1040383217,20220420,202204,2022,2022.3014,cre,CREE,NaN,NaN,cre,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220420234500,https://www.cjvr.com/2022/04/20/first-nations-...
125667,1040383218,20220420,202204,2022,2022.3014,cre,CREE,NaN,NaN,cre,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220420234500,https://www.cjvr.com/2022/04/20/first-nations-...
125668,1040383219,20220420,202204,2022,2022.3014,telOPP,TELUGU,NaN,NaN,tel,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220420234500,https://www.deccanchronicle.com/nation/politic...


Event data is highly denormalized with many redundancies for ease of querying and coded using a hierachical coding system called [CAMEO](http://data.gdeltproject.org/documentation/CAMEO.Manual.1.1b3.pdf) - Conflict and Mediation Event Observations

In [7]:
events[['EventCode', 'CAMEOCodeDescription']]

,EventCode,CAMEOCodeDescription
0,084,"Return, release, not specified below"
1,012,Make pessimistic comment
2,190,"Use conventional military force, not specifie..."
3,190,"Use conventional military force, not specifie..."
4,020,"Appeal, not specified below"
...,...,...
125665,060,"Engage in material cooperation, not spec below"
125666,073,Provide humanitarian aid
125667,090,"Investigate, not specified below"
125668,043,Host a visit


We can query and filter event data directly using the Pandas dataframe

In [8]:
# Find all events that were geolocated in Ukraine
uka_events = events[(events.ActionGeo_CountryCode == 'UP')]
uka_events

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
126,1039301016,20220414,202204,2022,2022.2849,NaN,NaN,NaN,NaN,NaN,...,4,"Kharkiv, Kharkivs'ka Oblast', Ukraine",UP,UP07,25036,49.9808,36.2527,-1041320,20220414013000,https://www.understandingwar.org/backgrounder/...
127,1039301017,20220414,202204,2022,2022.2849,NaN,NaN,NaN,NaN,NaN,...,4,"Kherson, Khersons'ka Oblast', Ukraine",UP,UP08,28557,46.6558,32.6178,-1041356,20220414013000,https://www.understandingwar.org/backgrounder/...
260,1039301150,20220414,202204,2022,2022.2849,CAN,CANADA,CAN,NaN,NaN,...,1,Ukraine,UP,UP,NaN,49.0000,32.0000,UP,20220414013000,https://www.castanet.net/news/Canada/365990/Se...
276,1039301166,20220414,202204,2022,2022.2849,CAN,CANADA,CAN,NaN,NaN,...,1,Ukraine,UP,UP,NaN,49.0000,32.0000,UP,20220414013000,https://www.castanet.net/news/Canada/365990/Se...
286,1039301176,20220414,202204,2022,2022.2849,CAN,CANADIAN,CAN,NaN,NaN,...,1,Ukraine,UP,UP,NaN,49.0000,32.0000,UP,20220414013000,https://www.castanet.net/news/Canada/365990/Se...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125624,1040383175,20220420,202204,2022,2022.3014,USAGOV,THE WHITE HOUSE,USA,NaN,NaN,...,1,Ukraine,UP,UP,NaN,49.0000,32.0000,UP,20220420234500,https://www.hellenicshippingnews.com/u-s-crude...
125625,1040383176,20220420,202204,2022,2022.3014,USAGOV,JOE BIDEN,USA,NaN,NaN,...,1,Ukraine,UP,UP,NaN,49.0000,32.0000,UP,20220420234500,https://www.agassizharrisonobserver.com/news/t...
125648,1040383199,20220420,202204,2022,2022.3014,VAT,VATICAN,VAT,NaN,NaN,...,1,Ukraine,UP,UP,NaN,49.0000,32.0000,UP,20220420234500,http://www.icatholic.org/article/christs-resur...
125651,1040383202,20220420,202204,2022,2022.3014,VAT,VATICAN,VAT,NaN,NaN,...,1,Ukraine,UP,UP,NaN,49.0000,32.0000,UP,20220420234500,http://www.icatholic.org/article/christs-resur...


So about 50K of 700K events last week were coded as happening in Ukraine, not surprising given recent events. Many of those related to use of military force.

In [9]:
# CAMEO code 190 denotes 'use of military force'
uka_events[uka_events.EventCode.str.startswith('190')]

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
568,1039301458,20220414,202204,2022,2022.2849,GOV,GOVERNOR,NaN,NaN,NaN,...,4,"Kharkiv, Kharkivs'ka Oblast', Ukraine",UP,UP07,25036,49.9808,36.2527,-1041320,20220414013000,http://www.koreatimes.co.kr/www/world/2022/04/...
894,1039301784,20220414,202204,2022,2022.2849,RUS,RUSSIAN,RUS,NaN,NaN,...,5,"Kherson Oblast, Khersons'ka Oblast', Ukraine",UP,UP08,28550,46.5000,34.0000,-1041362,20220414013000,https://www.understandingwar.org/backgrounder/...
913,1039301803,20220414,202204,2022,2022.2849,RUS,RUSSIAN,RUS,NaN,NaN,...,4,"Kherson, Khersons'ka Oblast', Ukraine",UP,UP08,28557,46.6558,32.6178,-1041356,20220414013000,https://www.understandingwar.org/backgrounder/...
915,1039301805,20220414,202204,2022,2022.2849,RUS,RUSSIA,RUS,NaN,NaN,...,5,"Kherson Oblast, Khersons'ka Oblast', Ukraine",UP,UP08,28550,46.5000,34.0000,-1041362,20220414013000,https://www.understandingwar.org/backgrounder/...
916,1039301806,20220414,202204,2022,2022.2849,RUS,RUSSIAN,RUS,NaN,NaN,...,4,"Rubizhne, Luhans'ka Oblast', Ukraine",UP,UP14,25090,49.0123,38.3797,-1052568,20220414013000,https://www.understandingwar.org/backgrounder/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118792,1040373135,20220420,202204,2022,2022.3014,RUS,RUSSIAN,RUS,NaN,NaN,...,4,"Vadym, Khersons'ka Oblast', Ukraine",UP,UP08,28553,46.1827,33.5971,-1057325,20220420221500,https://www.news8000.com/i/elderly-in-ukraine-...
118793,1040373136,20220420,202204,2022,2022.3014,RUS,RUSSIAN,RUS,NaN,NaN,...,4,"Vadym, Khersons'ka Oblast', Ukraine",UP,UP08,28553,46.1827,33.5971,-1057325,20220420221500,https://www.news8000.com/i/elderly-in-ukraine-...
118874,1040373217,20220420,202204,2022,2022.3014,UKR,UKRAINE,UKR,NaN,NaN,...,4,"Chernihiv, Chernihivs'ka Oblast', Ukraine",UP,UP02,28554,51.5055,31.2849,-1037057,20220420221500,http://www.msn.com/en-us/news/world/a-bomb-sni...
118949,1040373292,20220420,202204,2022,2022.3014,USA,UNITED STATES,USA,NaN,NaN,...,4,"Kyiv, Kyyiv, Misto, Ukraine",UP,UP12,28554,50.4333,30.5167,-1044367,20220420221500,http://www.msn.com/en-us/news/world/as-a-new-u...


In [10]:
# Import Folium to plot these military force events on a map
import folium
folium.Map(
    location=[48., 31.], 
    tiles="Stamen Toner",
    zoom_start=6
)

In [11]:
uka_map = folium.Map(
    location=[48., 31.], 
    #tiles="Stamen Toner",
    zoom_start=6
)
uka_map
uka_events_sample = uka_events[uka_events.EventCode.str.startswith('190')].sample(n=100)
for r in uka_events_sample.itertuples():
    m = folium.Marker(location=[r.ActionGeo_Lat, r.ActionGeo_Long],
                      icon=folium.Icon(color="red", icon="fire", prefix="glyphicon"),
                      tooltip=str(r.Actor1CountryCode) + '->' + str(r.EventCode) + ' ' +  str(r.CAMEOCodeDescription) + '->' + str(r.Actor2CountryCode) +' on ' + str(r.SQLDATE)
                     )
    m.add_to(uka_map)
uka_map

### Shaping tabular data into graph vertices and edges

The GDELT data schema is 'flat' and designed for easy of tabular querying and grouping. To be able to do graph and network queries it needs to be shaped.

In [12]:
from data.etl import shape_event_actor_vertices
events_vertices, actor1_vertices, actor2_vertices = shape_event_actor_vertices(uka_events_sample)

Hashing Actor1 ID:   0%|          | 0/100 [00:00<?, ?row/s]

Hashing Actor2 ID:   0%|          | 0/100 [00:00<?, ?row/s]

We create unique IDs for actors that can be linked to actions. We hash individual actor fields together to create a unique ID for each actor and then drop all the other actor fields from event data.

In [13]:
events_vertices

,ID,Actor1ID,Actor2ID,Date,IsRoot,MonthYear,Year,FractionDate,Actor1Religion1Code,Actor1Religion2Code,...,Actor2Geo_ADM1Code,Actor2Geo_ADM2Code,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
58320,1040279291,iHs/7zpbWweQ7HyWMCUOKSYtjeY=,BHlP81Q8yo2ECEO+A3HQvOeiJpY=,2022-04-20,False,202204,2022,2022.3014,NaN,NaN,...,UP14,28556,4,Kharkiv; Kharkivs'ka Oblast'; Ukraine,UP,49.9808,36.2527,-1041320,20220420114500,https://www.msn.com/en-us/news/world/russias-a...
48140,1040265068,dONASRx0Zy15mLVHFerS53DGqvc=,oNVaUzXhkpnRqqUT1e6yOeB4hLo=,2022-04-20,True,202204,2022,2022.3014,NaN,NaN,...,UP07,25036,4,Kharkiv; Kharkivs'ka Oblast'; Ukraine,UP,49.9808,36.2527,-1041320,20220420101500,https://www.presstv.ir/Detail/2022/04/20/68066...
40970,1040255296,v8sp9cq+wHnSUTOWTLwsFqxj6pY=,Bz/D8gHIXagQ62++bh1sT09EE9Q=,2022-04-20,False,202204,2022,2022.3014,NaN,NaN,...,UP07,25036,4,Kharkiv; Kharkivs'ka Oblast'; Ukraine,UP,49.9808,36.2527,-1041320,20220420090000,https://omaha.com/news/article_784d6fc3-d424-5...
51830,1039937236,UK6Dj7YHjBWh4iLV9jZdvYlKQGo=,t/RQ5wPBAKIEmMQJP+mzTCWyl6U=,2022-04-18,False,202204,2022,2022.2959,NaN,NaN,...,RS48,25106,4,Kherson; Khersons'ka Oblast'; Ukraine,UP,46.6558,32.6178,-1041356,20220418140000,https://sofrep.com/news/bloodbath-in-ukraine-8...
10060,1039638169,RR+Iy3oQSCGaM8z1aRcRsVRPgQA=,0juOXaLroTNODaUDjYStMzoyKWQ=,2022-04-16,True,202204,2022,2022.2904,NaN,NaN,...,UP00,25090,4,Donbas; Ukraine (general); Ukraine,UP,48.5000,38.5000,-1038077,20220416000000,https://apnews.com/article/russia-ukraine-kyiv...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29943,1040229948,1+VpnEK6JAOcDXUg+Wi6FLQuOHM=,2eJthRTR/8uSK2zPNecAoQwHw9E=,2022-04-20,False,202204,2022,2022.3014,NaN,NaN,...,UP00,25090,4,Luhansk; Luhans'ka Oblast'; Ukraine,UP,48.5670,39.3171,-1045160,20220420051500,https://www.theguardian.com/world/2022/apr/20/...
38825,1040063798,TT8E4ff/E74UloJv8jqG6/TMJ1w=,IPptSVeOo81DUs3YPINY6QcMTjQ=,2022-04-19,False,202204,2022,2022.2986,NaN,NaN,...,UP14,28556,4,Lymanske; Odes'ka Oblast; Ukraine,UP,46.6557,29.9672,-1044754,20220419084500,https://www.sheppnews.com.au/world/russia-step...
62092,1039571188,4ct6eVpwFT8dGMzBZAQUHhRk8vs=,JYvltOPrhyCFjFL+Iqd43hTRQWI=,2022-04-15,True,202204,2022,2022.2877,NaN,NaN,...,UP12,28554,4,Kyiv; Kyyiv; Misto; Ukraine,UP,50.4333,30.5167,-1044367,20220415134500,https://www.abc15.com/news/national/russia-ukr...
31559,1039904750,PdW2+wiyjAcN2y1x3ynxRHMp9QY=,tlifxqsNyCzxIJnRwtQKuZToQQw=,2022-03-19,False,202203,2022,2022.2164,NaN,NaN,...,NaN,NaN,4,Kherson; Khersons'ka Oblast'; Ukraine,UP,46.6558,32.6178,-1041356,20220418094500,https://www.thesun.co.uk/news/18294062/putin-l...


The actor information is now stored as separate entities that can be linked to actions.

In [51]:
actor1_vertices

,ActorID,ActorCode,ActorName,ActorCountryCode,ActorKnownGroupCode,ActorEthnicCode,ActorGeo_Type,ActorGeo_FullName,ActorGeo_CountryCode,ActorGeo_Lat,ActorGeo_Long,ActorGeo_ADMCode,ActorGeo_FeatureID
58320,iHs/7zpbWweQ7HyWMCUOKSYtjeY=,UKR,KHARKIV,UKR,NaN,NaN,4,"Kramatorsk, Donets'ka Oblast', Ukraine",UP,48.7230,37.5563,UP0528549,-1043300
48140,dONASRx0Zy15mLVHFerS53DGqvc=,MIL,COMMANDER,NaN,NaN,NaN,4,"Kharkiv, Kharkivs'ka Oblast', Ukraine",UP,49.9808,36.2527,UP0725036,-1041320
40970,v8sp9cq+wHnSUTOWTLwsFqxj6pY=,RUS,RUSSIAN,RUS,NaN,NaN,4,"Kharkiv, Kharkivs'ka Oblast', Ukraine",UP,49.9808,36.2527,UP0725036,-1041320
51830,UK6Dj7YHjBWh4iLV9jZdvYlKQGo=,RUS,RUSSIA,RUS,NaN,NaN,4,"Moskva, Moskva, Russia",RS,55.7522,37.6156,RS4825106,-2960561
10060,RR+Iy3oQSCGaM8z1aRcRsVRPgQA=,RUS,RUSSIAN,RUS,NaN,NaN,4,"Moscow, Moskva, Russia",RS,55.7522,37.6156,RS4825106,-2960561
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29943,1+VpnEK6JAOcDXUg+Wi6FLQuOHM=,UKR,UKRAINE,UKR,NaN,NaN,4,"Donbas, Ukraine (general), Ukraine",UP,48.5000,38.5000,UP0025090,-1038077
38825,TT8E4ff/E74UloJv8jqG6/TMJ1w=,UKR,UKRAINE,UKR,NaN,NaN,4,"Lymanske, Odes'ka Oblast, Ukraine",UP,46.6557,29.9672,UP1741206,-1044754
62092,4ct6eVpwFT8dGMzBZAQUHhRk8vs=,RUSGOVMIL,RUSSIA,RUS,NaN,NaN,4,"Moscow, Moskva, Russia",RS,55.7522,37.6156,RS4825106,-2960561
31559,PdW2+wiyjAcN2y1x3ynxRHMp9QY=,MIL,LIEUTENANT GENERAL,NaN,NaN,NaN,4,"Moscow, Moskva, Russia",RS,55.7522,37.6156,RS4825106,-2960561


We can visualize this data using Graphistry. First let's 'flatten' the graph schema so we only have one type of node and edge

In [ ]:
from data.etl import flatten_event_actor_vertices
nodes, edges = flatten_event_actor_vertices_flat(events_vertices, actor1_vertices, actor2_vertices)
nodes

In [15]:
# The following is used running inside Colab to pull env variables from a file on your GDrive
if IN_COLAB:
  print('Installing colab-env whiich pull env variable values from a file called vars.env on your GDrive.')
  print(subprocess.run('pip install colab-env --upgrade', text=True, shell=True, check=True, capture_output=True).stdout)

In [16]:
# Begin the authorization process for GDrive
if IN_COLAB:
    import colab_env

In [37]:
# Start using Graphistry, you'll need GRAPHISTRY_USER and GRAPHISTRY_PASS env variables
# If not using vars.env or not in Colab you'll have to set it manually
# os.envriron[''GRAPHISTRY_USER'] = mygruser
# os.envriron[''GRAPHISTRY_PASS'] = mygrpass

assert 'GRAPHISTRY_USER' in os.environ and 'GRAPHISTRY_PASS' in os.environ
from graphistry import graphistry
graphistry.register(api=3, username=os.environ['GRAPHISTRY_USER'], password=os.environ['GRAPHISTRY_PASS'], protocol='https', server='hub.graphistry.com')

In [50]:
# Plot UP events using Graphistry
g = graphistry.bind(source="src", destination="dest", edge_title="date", node="node_id")
g.edges(edges).nodes(nodes).plot()